## Скачать нейросети ResNet и написать короткую процедуру для предсказания класса изображения

Этот код загружает предварительно обученную модель ResNet50, использует её для предсказания классов изображения и выводит топ-3 наиболее вероятных класса. Убедитесь, что путь к изображению указан правильно в вызове функции predict_class.

In [2]:
# Установите необходимые библиотеки:

# pip install tensorflow keras pillow


In [3]:
# Загрузите предварительно обученную модель ResNet50 из Keras:

from keras.applications.resnet50 import ResNet50

model = ResNet50(weights='imagenet')

102967424/102967424 ━━━━━━━━━━━━━━━━━━━━ 135s 1us/step


In [25]:
# Напишите функцию для предсказания класса изображения:

from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np


def predict_class(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    preds = model.predict(x)
    print('Predicted:', decode_predictions(preds, top=3)[0])

In [26]:
# Используйте функцию predict_class для предсказания класса изображения:

predict_class('nfs-need-for-speed-police-airplanes.jpeg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
Predicted: [('n04037443', 'racer', 0.9335238), ('n04285008', 'sports_car', 0.050977875), ('n02974003', 'car_wheel', 0.0060688853)]


---

## Скачать нейросеть BERT для лингвистических задач и реализовать процедуру классификации текстов (без оглядки на качество классификации)

Этот код предоставляет базовую структуру для работы с BERT и классификации текстов. Вы можете адаптировать его под свои нужды, изменяя параметры модели, методы предварительной обработки и классификации.

Он позволяет классифицировать тексты на основе их семантического содержания, представленного в виде эмбеддингов, полученных от модели BERT/RuBERT. Такой подход может быть использован во многих задачах обработки естественного языка, таких как анализ тональности, классификация текстов по категориям и многое другое.

In [14]:
# Установите библиотеку transformers, если у вас её ещё нет:

# pip install transformers


In [21]:
# Импортируйте необходимые модули и загрузите модель BERT. 
# В этом примере используется англоязычная версия BERT (bert-base-uncased), 
# но вы можете выбрать любую другую доступную модель.

# from transformers import BertTokenizer, BertModel

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertModel.from_pretrained('bert-base-uncased')

In [2]:
# Чтобы использовать RuBERT вместо стандартной модели BERT в вашем коде, 
# вам нужно заменить 'bert-base-uncased' на идентификатор модели RuBERT. 
# Для этого воспользуйтесь моделью DeepPavlov/rubert-base-cased.

from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
model = AutoModel.from_pretrained('DeepPavlov/rubert-base-cased')

# Здесь мы используем AutoTokenizer и AutoModel из библиотеки transformers, 
# чтобы автоматически определить тип токенизатора и модели, соответствующий 
# указанному идентификатору. Это позволяет легко переключаться между различными моделями, включая RuBERT.

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [29]:
# Предварительная обработка текста
# Преобразуйте ваши тексты в тензоры, используя токенизатор BERT.

texts = ["Наследственные заболевания сетчатки (НЗС), являясь орфанными заболеваниями, имеют высокую социальную значимость ввиду выраженных нарушений зрения и инвалидности пациентов в детском и трудоспособном возрасте. Известно более 100 нозологических форм НЗС, которые вызываются различными дефектами, обнаруженными примерно в 270 генах.",
         "Автомобилисты, приходящие из поисковых систем на сайт магазина автозапчастей по запросам «как избежать перегрева двигателя» и «как защитить радиатор автомобиля от повреждений», найдут ответ в тексте по установке фальшрадиаторной сетки. Вероятно, и закажут запчасть именно на этом сайте."]
inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)

In [30]:
# Получение эмбеддингов из модели BERT
# Выполните прямой проход через модель BERT, чтобы получить эмбеддинги для ваших текстов.

outputs = model(**inputs)
last_hidden_states = outputs.last_hidden_state

In [43]:
# Классификация текстов
# Для простейшей классификации можно использовать среднее значение эмбеддингов по последней размерности 
# и затем применить любой классификатор. Например, используйте логистическую регрессию из sklearn.

import torch
from sklearn.linear_model import LogisticRegression

# Среднее значение эмбеддингов
embeddings = last_hidden_states.mean(dim=1).detach().numpy()

# Пример меток классов
labels = [0, 1]  # Замените на реальные метки

# Обучение классификатора
classifier = LogisticRegression()
classifier.fit(embeddings, labels)

# Предсказание классов для новых текстов
new_texts = [
    "ракета"
    ]
new_inputs = tokenizer(new_texts, return_tensors="pt",
                       padding=True, truncation=True)
with torch.no_grad():
    new_outputs = model(**new_inputs)
    new_embeddings = new_outputs.last_hidden_state.mean(dim=1).numpy()
predictions = classifier.predict(new_embeddings)

print("Класс:", predictions)

Класс: [1]
